# Tweets from bioRxiv and medRxiv

Find and display events from the CSHL DOI prefix 10.1101 from the last 7 days, and find the most tweeted preprints.

In [1]:
import sys
sys.path.append('../')
sys.path.insert(0, '..')

import pandas as pd # data analysis library
import json
import mrced2 # module to run event data queries
import os # some file manipulations
import math # some number manipulations
import altair.vegalite.v3 as alt # some data visualizations
from IPython.display import Markdown as md # some markdown manipulations
from datetime import datetime, date, timedelta # some date manipulations

In [2]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = (date.today() - timedelta(days = 7)).strftime('%Y-%m-%d')
end_date = date.today()

In [3]:
ed = mrced2.eventData(email = email)
ed.buildQuery({'obj-id.prefix' : prefix, 'source': 'twitter', 'rows': 0,'from-occurred-date' : start_date, 'until-occurred-date' : end_date})

https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&obj-id.prefix=10.1101&source=twitter&rows=0&from-occurred-date=2021-05-31&until-occurred-date=2021-06-07


In [4]:
ed.runQuery(retry = 5)

Event Data query started...
API query complete  200
output file written to 1101/tweets.json


In [5]:
pages = math.ceil(ed.events.getHits() / 1000)

8975 events found


In [6]:
email = "info@front-matter.io"
prefix = "10.1101"
start_date = date.today() - timedelta(days = 7)
end_date = date.today()

# find the all result pages for the search
ed = mrced2.eventData(email = email)
ed.getAllPages(pages, {'rows': 1000, 'obj-id.prefix' : prefix, 'source': 'twitter', 'from-occurred-date' : start_date, 'until-occurred-date' : end_date}, fileprefix = '1101/tweets_') 

https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-05-31&until-occurred-date=2021-06-07
Event Data query started...
API query complete  200
output file written to 1101/tweets_0000.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=4c443194-4eb3-40a3-99a0-8118b5195a16&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-05-31&until-occurred-date=2021-06-07
Event Data query started...
API query complete  200
output file written to 1101/tweets_0001.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=289336dd-ee7f-460b-8cde-6902f7b48bff&rows=1000&obj-id.prefix=10.1101&source=twitter&from-occurred-date=2021-05-31&until-occurred-date=2021-06-07
Event Data query started...
API query complete  200
output file written to 1101/tweets_0002.json
https://api.eventdata.crossref.org/v1/events?mailto=Anonymous&cursor=1d099dfc-9db6-4fc9-86bd-90c63b2d808e&rows=

Initialisation to look at the properties of the results.

In [7]:
jd1 = mrced2.eventRecord() # instance of a class to interpret the events
files = os.listdir('1101') # get all the filenames

jd1.mergeJsons(files, folder = '1101') # load the json event data from multiple files

failed to load preprint_tweets_2021-05-10.csv
failed to load preprint_tweets_2021-05-17.csv
failed to load .DS_Store
failed to load .gitkeep
failed to load tweets_sorted.json
failed to load preprint_tweets_2021-05-18.csv
failed to load preprint_tweets_2021-05-24.csv
failed to load preprint_tweets_2021-05-31.csv
failed to load preprint_tweets_2021-05-09.csv
failed to load preprint_tweets_2021-05-08.csv
output file written to 1101/tweets.json


In [8]:
js = json.load(open("1101/tweets.json"))
df = pd.json_normalize(js, record_path = ['message', 'events'])
gdf = df.groupby(['obj_id']).size().reset_index(name='count').sort_values('count', ascending=False)
cdf = gdf[gdf['count'] >= 5]

In [9]:
email = "info@front-matter.io"
rest = mrced2.restApi(email = email)

data = []

for index, row in cdf.iterrows():
    rest.runQuery(row)
    if rest.work is not None and date.fromisoformat(rest.work["posted"]) >= start_date:
        data.append(rest.work)
    
tdf = pd.DataFrame(data, columns=['doi','tweets','archive','subject-area','covid','title','authors','abstract','posted'])
tdf.to_csv('1101/preprint_tweets_' + str(date.today().strftime('%Y-%m-%d')) + '.csv')

tdf.head(50)

REST API query started for 10.1101/2021.03.04.430128...
REST API query complete  200
REST API query started for 10.1101/2021.05.19.21257334...
REST API query complete  200
REST API query started for 10.1101/2021.03.12.21253490...
REST API query complete  200
REST API query started for 10.1101/2021.05.30.21257971...
REST API query complete  200
REST API query started for 10.1101/2020.12.12.422516...
REST API query complete  200
REST API query started for 10.1101/2020.11.15.383323...
REST API query complete  200
REST API query started for 10.1101/2021.05.28.446163...
REST API query complete  200
REST API query started for 10.1101/2020.01.30.927871...
REST API query complete  200
REST API query started for 10.1101/2021.01.21.21250266...
REST API query complete  200
REST API query started for 10.1101/2021.05.11.21257037...
REST API query complete  200
REST API query started for 10.1101/2021.01.27.21250617...
REST API query complete  200
REST API query started for 10.1101/2021.03.05.2125296

,doi,tweets,archive,subject-area,covid,title,authors,abstract,posted
0,10.1101/2021.05.30.21257971,703,medRxiv,Infectious Diseases (except HIV/AIDS),True,Heterologous ChAdOx1 nCoV-19 and BNT162b2 prim...,"[{'name': 'Rüdiger Groß'}, {'name': 'Michelle ...",<sec><title>Background</title><p>Heterologous ...,2021-06-01
1,10.1101/2021.05.28.21257993,161,medRxiv,Genetic and Genomic Medicine,True,The UGT2A1/UGT2A2 locus is associated with COV...,"[{'name': 'Janie F. Shelton'}, {'name': 'Anjal...",<p>Loss of sense of smell is a characteristic ...,2021-05-31
2,10.1101/2021.06.01.446439,101,bioRxiv,Neuroscience,False,The human language system does not support mus...,"[{'name': 'Xuanyi Chen'}, {'name': 'Josef Affo...",<p>Language and music are two human-unique cap...,2021-06-01
3,10.1101/2021.06.02.446311,80,bioRxiv,Developmental Biology,False,Hybridization chain reaction enables a unified...,"[{'name': 'Maayan Schwarzkopf'}, {'name': 'Mik...",<p>RNA in situ hybridization (RNA-ISH) based o...,2021-06-02
4,10.1101/2021.05.31.445667,76,bioRxiv,Molecular Biology,True,Revealing the host antiviral protein ZAP-S as ...,"[{'name': 'Matthias M. Zimmer'}, {'name': 'Anu...",<p>Programmed ribosomal frameshifting (PRF) is...,2021-05-31
5,10.1101/2021.05.30.446357,69,bioRxiv,Microbiology,True,Complete protection by a single dose skin patc...,"[{'name': 'Christopher L.D. McMillan'}, {'name...",<p>SARS-CoV-2 has infected over 160 million pe...,2021-05-31
6,10.1101/2021.06.03.446934,64,bioRxiv,Bioinformatics,False,"eggNOG-mapper v2: Functional Annotation, Ortho...","[{'name': 'Carlos P. Cantalapiedra'}, {'name':...",<p>Even though automated functional annotation...,2021-06-03
7,10.1101/2021.05.31.446380,58,bioRxiv,Microbiology,False,An ancient divide in outer membrane tethering ...,"[{'name': 'Jerzy Witwinowski'}, {'name': 'Anna...",<p>Recent data support the hypothesis that Gra...,2021-05-31
8,10.1101/2021.05.31.445775,52,bioRxiv,Neuroscience,False,Mediobasal hypothalamic FKBP51 acts as a molec...,"[{'name': 'Alexander S. Häusl'}, {'name': 'Lea...",<p>The mediobasal hypothalamus (MBH) is the ce...,2021-05-31
9,10.1101/2021.05.31.446434,36,bioRxiv,Molecular Biology,False,Time-resolved cryo-EM visualizes ribosomal tra...,"[{'name': 'Christine E. Carbone'}, {'name': 'A...","<p>During translation, a conserved GTPase elon...",2021-05-31


### Tweets of bioRxiv and medRxiv preprints

In [10]:
num_rows = tdf['archive'].count()
num_covid = tdf['covid'].value_counts(ascending=True)[1]
num_biorxiv = tdf['archive'].value_counts(ascending=True)[1]
num_medrxiv = tdf['archive'].value_counts(ascending=True)[0]
end_date = date.today().strftime('%Y-%m-%d')
max_count = tdf['tweets'].max()

md('{} preprints (including {} covering SARS-CoV-2, {} from bioRxiv and {} from medRxiv) published in the last 7 days before {} had been tweeted at least 5 times (maximum {}).'.format(num_rows, num_covid, num_biorxiv, num_medrxiv, end_date, max_count))

53 preprints (including 14 covering SARS-CoV-2, 43 from bioRxiv and 10 from medRxiv) published in the last 7 days before 2021-06-07 had been tweeted at least 5 times (maximum 703).